## Tuning the hyperparameters of a neural network using EasyVVUQ

In this tutorial we will use the EasyVVUQ `GridSampler` to perform a grid search on the hyperparameters of a simple Keras neural network model, trained to recognize hand-written digits. This is the famous MNIST data set, of which 4 input features (of size 28 x 28) are show below. These are fed into a standard feed-forward neural network, which will predict the label 0-9.

**Note**: This tutorial always runs on the localhost. One possibility of performing the grid search on a remote supercomputer involves the use of FabSim, see the `hyperparameter_tuning_tutorial_with_fabsim.ipynb` notebook tutorial.

The (Keras) neural network script is located in `mnist/keras_mnist.template`, which will form the input template for the EasyVVUQ encoder. We will assume you are familiar with the basic EasyVVUQ building blocks. If not, you can look at the [basic tutorial](https://github.com/UCL-CCS/EasyVVUQ/blob/dev/tutorials/basic_tutorial.ipynb).

![](mnist/mnist_feats.png)

We need EasyVVUQ, TensorFlow and the TensorFlow data sets to execute this tutorial. If you need to install these, uncomment the corresponding line below.

In [1]:
# !pip install easyvvuq
# !pip install tensorflow
# !pip install tensorflow_datasets

In [2]:
import easyvvuq as uq
import os
import numpy as np
from easyvvuq.actions import CreateRunDirectory, Encode, Decode, ExecuteLocal, Actions

We now set some flags:

In [3]:
# Work directory, where the EasyVVUQ directory will be placed
WORK_DIR = '/tmp'
# target output filename generated by the code
TARGET_FILENAME = 'output.csv'
# EasyVVUQ campaign name
CAMPAIGN_NAME = 'grid_test'

As is standard in EasyVVUQ, we now define the parameter space. In this case these are 4 hyperparameters. There is one hidden layer with `n_neurons` neurons, a Dropout layer after the input and hidden layer, with dropout probability `dropout_prob_in` and `dropout_prob_hidden` respectively. We made the `learning_rate` tuneable as well.

In [4]:
params = {}
params["n_neurons"] = {"type":"integer", "default": 32}
params["dropout_prob_in"] = {"type":"float", "default": 0.0}
params["dropout_prob_hidden"] = {"type":"float", "default": 0.0}
params["learning_rate"] = {"type":"float", "default": 0.001}

These 4 hyperparameter appear as flags in the input template `mnist/keras_mnist.template`. Typically this is generated from an input file used by some simualtion code. In this case however, `mnist/keras_mnist.template` is directly our Python script, with the hyperparameters replaced by flags. For instance:

```python
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dropout($dropout_prob_in),
  tf.keras.layers.Dense($n_neurons, activation='relu'),
  tf.keras.layers.Dropout($dropout_prob_hidden),
  tf.keras.layers.Dense(10)
])
```

is simply the neural network construction part with flags for the dropout probabilities and the number of neurons in the hidden layer. The encoder reads the flags and replaces them with numeric values, and it subsequently writes the corresponding `target_filename=hyper_param_tune.py`:

In [5]:
encoder = uq.encoders.GenericEncoder('./mnist/keras_mnist.template', target_filename='hyper_param_tune.py')

What follows are standard steps in setting up an EasyVVUQ Campaign

In [6]:
# execute the runs locally
execute = ExecuteLocal('python3 hyper_param_tune.py')

# decode the output CSV files, with stored training and test accuracy values
output_columns = ["accuracy_train", "accuracy_test"]
decoder = uq.decoders.SimpleCSV(target_filename=TARGET_FILENAME, output_columns=output_columns)

# actions are 1) create run dirs, 2) encode input template, 3) execute runs, 4) decode output files
actions = Actions(CreateRunDirectory(root='/tmp', flatten=True), Encode(encoder), execute, Decode(decoder))

# create the EasyVVUQ main campaign object
campaign = uq.Campaign(
    name=CAMPAIGN_NAME,
    work_dir=WORK_DIR,
)

# add the param definitions and actions to the campaign
campaign.add_app(
    name=CAMPAIGN_NAME,
    params=params,
    actions=actions
)

As with the uncertainty-quantification (UQ) samplers, the `vary` is used to select which of the `params` we actually vary. Unlike the UQ samplers we do not specify an input probability distribution. This being a grid search, we simply specify a list of values for each hyperparameter. Parameters not in `vary`, but with a flag in the template, will be given the default value specified in `params`.

In [7]:
vary = {"n_neurons": [64, 128], "learning_rate": [0.005, 0.01, 0.015]}

**Note:** we are mixing integer and floats in the `vary` dict. Other data types (string, boolean) can also be used.

The `vary` dict is passed to the `Grid_Sampler`. As can be seen, it created a tensor product of all 1D points specified in `vary`. If a single tensor product is not useful (e.g. because it creates combinations of parameters that do not makes sense), you can also pass a list of different `vary` dicts. For even more flexibility you can also write the required parameter combinations to a CSV file, and pass it to the `CSV_Sampler` instead.

In [8]:
# create an instance of the Grid Sampler
sampler = uq.sampling.Grid_Sampler(vary)

# Associate the sampler with the campaign
campaign.set_sampler(sampler)

# print the points
print("There are %d points:" % (sampler.n_samples()))
sampler.points

There are 6 points:


[array([[64, 0.005],
        [64, 0.01],
        [64, 0.015],
        [128, 0.005],
        [128, 0.01],
        [128, 0.015]], dtype=object)]

Run the `actions` (create directories with `hyper_param_tune.py` files in it)

In [ ]:
###############################
# execute the defined actions #
###############################

campaign.execute().collate()

2023-03-02 11:34:46.627885: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-02 11:34:46.628054: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-02 11:34:46.630651: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the ap

2023-03-02 11:34:52.172878: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-02 11:34:52.173245: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-02 11:34:52.173292: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (wouter-XPS-13-7390): /proc/driver/nvidia/version does not exist
2023-03-02 11:34:52.175002: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-02 11:34:52.177382: W tensorflow/compiler/xla/stream_exec

Epoch 1/6
Epoch 1/6
Epoch 1/6
Epoch 1/6
Epoch 1/6
Epoch 1/6
469/469 [==============================] - 15s 13ms/step - loss: 0.4033 - sparse_categorical_accuracy: 0.8792 - val_loss: 0.2163 - val_sparse_categorical_accuracy: 0.9375
Epoch 2/6
469/469 [==============================] - 15s 9ms/step - loss: 0.4464 - sparse_categorical_accuracy: 0.8710 - val_loss: 0.2590 - val_sparse_categorical_accuracy: 0.9264
Epoch 2/6
469/469 [==============================] - 16s 14ms/step - loss: 0.4242 - sparse_categorical_accuracy: 0.8773 - val_loss: 0.2251 - val_sparse_categorical_accuracy: 0.9334
Epoch 2/6
469/469 [==============================] - 16s 14ms/step - loss: 0.3825 - sparse_categorical_accuracy: 0.8884 - val_loss: 0.2041 - val_sparse_categorical_accuracy: 0.9407
Epoch 2/6
469/469 [==============================] - 3s 6ms/step - loss: 0.1926 - sparse_categorical_accuracy: 0.9432 - val_loss: 0.1692 - val_sparse_categorical_accuracy: 0.9502
Epoch 3/6
469/469 [=============================

460/469 [============================>.] - ETA: 0s - loss: 0.1721 - sparse_categorical_accuracy: 0.9512

In [ ]:
data_frame = campaign.get_collation_result()
data_frame

Display the hyperparameters with the maximum test accuracy

In [ ]:
print("Best hyperparameters with %.2f%% test accuracy:" % (data_frame['accuracy_test'].max().values * 100,))
data_frame.loc[data_frame['accuracy_test'].idxmax()][vary.keys()]